# Azure Queue Storage Demo

## Preparation

In [23]:
#r "nuget: Azure.Storage.Queues"

Installed Packages Azure.Storage.Queues, 12.23.0

Import namespaces:

In [24]:
using Azure.Storage.Queues;
using Azure.Storage.Queues.Models;

Setup connection string:

In [ ]:
var connectionString = "DefaultEndpointsProtocol=https;AccountName=ztechtest;AccountKey=XXX;EndpointSuffix=core.windows.net";

## Create and Initialize Queue Client
This step creates a queue client and initializes the queue.

In [26]:
// Name of the queue
var queueName = "my-queue";

// Create the queue client
var queueClient = new QueueClient(connectionString, queueName);

Console.WriteLine($"Creating queue '{queueName}'...");

// Create the queue if it doesn't exist
await queueClient.CreateIfNotExistsAsync();

if (queueClient.Exists())
{
    Console.WriteLine($"Queue '{queueName}' is ready.");
}
else
{
    Console.WriteLine($"Failed to create queue '{queueName}'.");
}

Creating queue 'my-queue'...
Queue 'my-queue' is ready.


## Add Messages to the Queue
Now, let's add some messages to the queue.

In [27]:
for (int i = 1; i <= 5; i++)
{
    await queueClient.SendMessageAsync($"Message #{i}");
}

Console.WriteLine("Messages added to the queue.");

Messages added to the queue.


## Retrieve and process messages

In [28]:
while (true)
{
    // Receive a message from the queue
    var msgReply = await queueClient.ReceiveMessageAsync(visibilityTimeout: TimeSpan.FromSeconds(30));

    // Check if a message was received; if not, queue is empty and exit the loop
    if (msgReply.Value == null) break;
    var msg = msgReply.Value;

    // Show message details
    msg.Display();

    // Delete the message from the queue
    await queueClient.DeleteMessageAsync(msg.MessageId, msg.PopReceipt);
}
Console.WriteLine("All messages received and deleted from the queue.");

Azure.Storage.Queues.Models.QueueMessage MessageId 4b9337ca-3910-4801-947b-4496976b5ea3 PopReceipt AgAAAAMAAAAAAAAAnuJK51MQ3AE= MessageText Message #1 Body Message #1 NextVisibleOn 2025-08-18 15:22:26Z InsertedOn 2025-08-18 15:20:12Z ExpiresOn 2025-08-25 15:20:12Z DequeueCount 1

Azure.Storage.Queues.Models.QueueMessage MessageId 45863ed8-8d0b-42c2-855e-d24dcb22a9a2 PopReceipt AgAAAAMAAAAAAAAApd9R51MQ3AE= MessageText Message #2 Body Message #2 NextVisibleOn 2025-08-18 15:22:26Z InsertedOn 2025-08-18 15:20:12Z ExpiresOn 2025-08-25 15:20:12Z DequeueCount 1

Azure.Storage.Queues.Models.QueueMessage MessageId 59183d12-cf32-4625-a611-cc492e08831d PopReceipt AgAAAAMAAAAAAAAAs9xY51MQ3AE= MessageText Message #3 Body Message #3 NextVisibleOn 2025-08-18 15:22:26Z InsertedOn 2025-08-18 15:20:12Z ExpiresOn 2025-08-25 15:20:12Z DequeueCount 1

Azure.Storage.Queues.Models.QueueMessage MessageId 463f9e41-2738-408d-a5ed-8759c5fd7e4c PopReceipt AgAAAAMAAAAAAAAA7otf51MQ3AE= MessageText Message #4 Body Message #4 NextVisibleOn 2025-08-18 15:22:26Z InsertedOn 2025-08-18 15:20:12Z ExpiresOn 2025-08-25 15:20:12Z DequeueCount 1

Azure.Storage.Queues.Models.QueueMessage MessageId 8a5b6ce5-d29b-45c6-b35b-d40ed97ed734 PopReceipt AgAAAAMAAAAAAAAAFWJm51MQ3AE= MessageText Message #5 Body Message #5 NextVisibleOn 2025-08-18 15:22:26Z InsertedOn 2025-08-18 15:20:12Z ExpiresOn 2025-08-25 15:20:12Z DequeueCount 1

All messages received and deleted from the queue.


The `visibilityTimeout` argument in `ReceiveMessageAsync` specifies how long the received message will be invisible to other consumers after being read. When a message is retrieved, it remains in the queue but is hidden for the duration of the timeout. If the message is not deleted within this period, it becomes visible again and can be processed by another consumer. This mechanism helps ensure that messages are not lost if processing fails, but also prevents multiple consumers from processing the same message simultaneously.

When message is returned into the queue and received again, its `DequeueCount` property is incremented. You may use this property to weed out _poison messages_, which can cause problems. If the same message fails processing multiple times, you can throw it away.

First, let's add some messages again. They will contain numbers, but some of them are invalid and would crash the processing code:

In [29]:
Console.Write("Generating: ");
for (int i = 1; i <= 20; i++) {
    var randomNumber = new Random().Next(1, 101);

    if (randomNumber <= 10) {
        // In 10% of the cases, intentionally generate invalid message 
        await queueClient.SendMessageAsync("Poison!");
        Console.Write("!");
    } else {
        await queueClient.SendMessageAsync(randomNumber.ToString());
        Console.Write(".");
    }
}

Generating: ......!.............

Now we will try to process the messages, but some will fail:

In [30]:
const int MaxRetry = 3;
const int MaxWaitTime = 10;

var sw = new System.Diagnostics.Stopwatch();

while(true) {
    // Receive a message from the queue
    var msgReply = await queueClient.ReceiveMessageAsync(visibilityTimeout: TimeSpan.FromSeconds(1));

    // Check if a message was received; if not, wait for a while
    if (msgReply.Value == null) {
        if(sw.Elapsed > TimeSpan.FromSeconds(MaxWaitTime)) {
            sw.Stop();
            Console.WriteLine("No messages in the queue, exiting.");
            break;
        }
        sw.Start();
        await Task.Delay(TimeSpan.FromSeconds(1));
        continue;
    }
    var msg = msgReply.Value;
    sw.Reset();

    // Check if the message wasn't already tried enough times
    if (msg.DequeueCount > MaxRetry) {
        // This is poison message, delete it
        Console.WriteLine($"Message #{msg.MessageId}: '{msg.Body}' is a poison message.");
        await queueClient.DeleteMessageAsync(msg.MessageId, msg.PopReceipt);
    } else {
        // Process the message
        try {
            Console.Write($"Processing message #{msg.MessageId}: '{msg.Body}'...");
            var x = int.Parse(msg.Body.ToString());
            await queueClient.DeleteMessageAsync(msg.MessageId, msg.PopReceipt);
            Console.WriteLine("OK");
        } catch (Exception ex) {
            Console.WriteLine($"Failed: {ex.Message}");
        }
    }
}

Processing message #87437259-500d-4e2a-8c1c-2f09f495c214: '74'...OK
Processing message #d72c5186-6918-4447-8d3c-2445dd392c50: '89'...OK
Processing message #bb337a7d-7a76-4632-a1f2-dd22086f2e11: '11'...OK
Processing message #f98d991d-52dd-4971-9569-07e6ee456ca4: '79'...OK
Processing message #eff1cbf0-decb-404a-bd52-c99817af1d17: '81'...OK
Processing message #d77e4bb8-30b1-426f-9835-87ce7df7ea7d: '85'...OK
Processing message #cee4a5e2-a967-4767-8018-0ba61484d9b4: 'Poison!'...Failed: The input string 'Poison!' was not in a correct format.
Processing message #9228c73a-f3a6-45c0-95f1-cca2ceb8ff7f: '39'...OK
Processing message #21e6f44f-4769-4e36-9280-60125bd52f11: '68'...OK
Processing message #8012ee7b-24ae-4e61-83a6-2dc1e6500b29: '58'...OK
Processing message #db9d2d3d-8177-4b2c-9dd5-d0350900618a: '17'...OK
Processing message #a2cf8c12-bb6c-480c-a303-04efc5a18e99: '100'...OK
Processing message #30008bd3-5773-4cfe-a842-b346b5372097: '44'...OK
Processing message #8a72d79d-c67c-4db2-8438-ea05a

## Clean-up

In [31]:
await queueClient.DeleteIfExistsAsync();
Console.WriteLine($"Queue '{queueName}' deleted.");

Queue 'my-queue' deleted.


Note: As with blob containers, if you delete a queue, it's just marked for deletion and it takes a while to be actually deleted. If you will try to recreate the queue immediatelly, it will fail.